In [ ]:
#get data
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!gdown https://drive.google.com/uc?id=1rfQKuAeiSztNW3FOFHUC9lSlt56O3LPm

Downloading...
From: https://drive.google.com/uc?id=1rfQKuAeiSztNW3FOFHUC9lSlt56O3LPm
To: /content/image_x64_RC1.tfrec
191MB [00:05, 37.9MB/s]


In [ ]:
!gdown https://drive.google.com/uc?id=1GciIey1d_ounjl8O1RDIRSxOxMLct6Av

Downloading...
From: https://drive.google.com/uc?id=1GciIey1d_ounjl8O1RDIRSxOxMLct6Av
To: /content/image_x64_RC2.tfrec
192MB [00:02, 78.8MB/s]


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib
import pydot
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
# tf.enable_eager_execution()
# import tensorflow.contrib.tpu.keras_to_tpu_model as tpu_Model

#### Data preprocessing pipeline

In [ ]:
gdrive_path = pathlib.Path('/content/gdrive/My Drive/MSR_Dataset/')
gdrive_path.exists()

True

In [ ]:
all_image_path = pd.read_csv(gdrive_path.joinpath('path_data_hd.csv').as_posix(), header=0).iloc[:, 1:].values

In [ ]:
# data preprocess 
''' 1. read_image
    2. apply log transforms
    3. normalize '''

def preprocess_ll_image(ll_image_path, pre_upscale=1, scales=[1, 10, 100, 300]):
    
    image = tf.read_file(ll_image_path)
    #decode and resize image
    s = 600
    try:
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)
    except:
        image = tf.image.decode_image(image, channels=3, dtype=tf.uint8)
        
    image = tf.image.resize_images(image, size=[s, s])
    
    # perform log transform and concatinate images to 12 channels default
    image = image_log_transform(image, pre_upscale, scales)
#     image.set_shape((s, s, 12))
    return(image)

def preprocess_gt_image(gt_image_path, pre_upscale=1, scales=[1, 10, 100, 300]):
    
    image = tf.read_file(gt_image_path)
    #decode and resize image
#     s=600
    try:
        image = tf.image.decode_png(image, channels=3, dtype=tf.uint8)
    except:
        image = tf.image.decode_image(image, channels=3, dtype=tf.uint8)
        
    image = tf.image.resize_images(image, size=[s, s])

    # perform log transform and concatinate images to 12 channels default
    image = normalize_image(image, 0, 1)
#     image.set_shape((s, s, 3))
    return(image)

def image_log_transform(image, pre_upscale, scales):
    #image transform log and concatinate
    image = tf.cast(image, tf.float32)
    image = tf.math.multiply(image, pre_upscale)
    multi_log_image_concat = None

    for scale in scales:
        transformed_log_image = normalize_image((tf.math.log1p(tf.multiply(image, tf.cast(scale, tf.float32)))/tf.math.log(tf.cast(scale+1, tf.float32))), 0, 1)
        if multi_log_image_concat==None:
            multi_log_image_concat = transformed_log_image
        else:
            multi_log_image_concat = tf.concat([multi_log_image_concat, transformed_log_image], axis=-1)
    return(multi_log_image_concat)

def normalize_image(image, newMin, newMax, dtype=tf.float32):

    #normalize image via Norm_minmax method
    image = tf.cast(image, tf.float32)
    min_image = tf.reduce_min(image)
    max_image = tf.reduce_max(image)
    reduced_image = tf.subtract(image, min_image)
    grad = tf.math.divide((newMin-newMax), (min_image-max_image))
    normalized_image = tf.math.add(tf.math.multiply(reduced_image, grad), newMin)
    normalized_image = tf.cast(normalized_image, dtype=dtype)
    return(normalized_image)


def parse_ll(x):
  result = tf.parse_tensor(x, out_type=tf.float32)
  result = tf.reshape(result, [600, 600, 12])
  return result


def parse_gt(x):
  result = tf.parse_tensor(x, out_type=tf.float32)
  result = tf.reshape(result, [600, 600, 3])
  return result

def parse_ds(x):
  result = tf.parse_tensor(x, out_type=tf.float32)
  result = tf.reshape(result, [64, 64, 15])
  return(result[:, :, :-3], result[:, :, -3:])

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_size = int(0.998*len(all_image_path))
# val_size = int(0.1*len(all_image_path))

Batch_size = 64
filenames = ['/content/image_x64_RC1.tfrec', '/content/image_x64_RC2.tfrec']

#image path to tf.data
ds_images = tf.data.TFRecordDataset(filenames, compression_type='ZLIB', num_parallel_reads=64)

#map preprocessing functions to image paths
ds_images = ds_images.map(parse_ds, num_parallel_calls=64)

#shufle dataset and repeat dataset
image_count = len(all_image_path)

# val_ds = ds_images.skip(train_size)

train_ds = ds_images.cache()
# val_ds = val_ds.cache(filename='./cache_val.tf-data')

train_ds = train_ds.shuffle(buffer_size=100)
# val_ds = val_ds.shuffle(buffer_size=5)

train_ds = train_ds.repeat()
# val_ds = val_ds.repeat()

train_ds = train_ds.batch(Batch_size)
# val_ds = val_ds.batch(Batch_size)

train_ds = train_ds.prefetch(20)
# val_ds = val_ds.prefetch(5)

print(train_ds)
# ds = ds.repeat()
# ds = ds.batch(Batch_size)
# ds = ds.prefetch(4)

# ds

<DatasetV1Adapter shapes: ((?, 64, 64, 12), (?, 64, 64, 3)), types: (tf.float32, tf.float32)>


In [ ]:
reg = keras.regularizers.l2(l=0.01)
#input layer
inputs = layers.Input(shape=(None, None, 12), dtype='float32')

#make better image by weighted average of this 12 channel image
conv1 = layers.Conv2D(filters=32, kernel_size=[1, 1], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(inputs)
conv2 = layers.Conv2D(filters=3, kernel_size=[3, 3], padding='same', kernel_regularizer=reg)(conv1)

#main leaning layer
conv3 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv2)
conv4 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv3)
conv5 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv4)
conv6 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv5)
conv7 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv6)
conv8 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv7)
conv9 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv8)
conv10 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv9)
conv11 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv10)
conv12 = layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(conv11)


#concatinate all smoothened images and subtract from conv2
concat_image = layers.concatenate(([conv3, conv4, conv5, conv6, conv7, conv8, conv9, conv10, conv11, conv12]), axis=-1)
conv_weighted_image = layers.Conv2D(filters=3, kernel_size=[1, 1], padding='same', kernel_regularizer=reg)(concat_image)

#subtract images
subtr_image = layers.subtract([conv2, conv_weighted_image])

#color restoration
color_restored_image = layers.Conv2D(filters=3, kernel_size=[1, 1], padding='same', activation=keras.activations.relu, kernel_regularizer=reg)(subtr_image)

model = Model(inputs=inputs, outputs=color_restored_image)

#### Final Model in Keras :-a high level api using backend tensorflow

In [ ]:
keras.utils.plot_model(model, to_file='modelLR_MR.png', show_shapes=True, show_layer_names=True, rankdir='TB')

In [ ]:
def loss(y_true, y_pred):
    y_pred = K.reshape(y_pred, [K.shape(y_pred)[0], K.shape(y_pred)[1]*K.shape(y_pred)[3]*K.shape(y_pred)[2]])
    y_true = K.reshape(y_true, [K.shape(y_true)[0], K.shape(y_true)[1]*K.shape(y_true)[3]*K.shape(y_true)[2]])
    loss = K.mean(K.sum(K.square(y_true-y_pred), axis=-1))
    return(loss)

def reduce_rate(epoch):
    if epoch>=100000:
        return(np.float32(0.000001))
    else:
        return(np.float32(0.0001))
    
lrate = keras.callbacks.LearningRateScheduler(reduce_rate)
checkpointer = keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/My Drive/MSR_Dataset/models_x64/weights.{epoch:02d}-{loss:.2f}.h5', monitor='loss', mode='auto',  verbose=1, save_best_only=True)
tbCallBack = keras.callbacks.TensorBoard(log_dir='/content/gdrive/My Drive/MSR_Dataset/Graph3', histogram_freq=0, write_graph=True, write_images=True)
callback_list = [lrate, checkpointer, tbCallBack]

In [ ]:
!mkdir /content/gdrive/My\ Drive/MSR_Dataset/models_x64/

mkdir: cannot create directory ‘/content/gdrive/My Drive/MSR_Dataset/models_x64/’: File exists


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(decay=0.0001),
                  loss=loss, 
                  metrics=['accuracy'])

In [ ]:
model.load_weights('/content/gdrive/My Drive/MSR_Dataset/model/model_E36.h5')

In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, None, None, 3 416         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, None, None, 3 867         conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, None, None, 3 896         conv2d_15[0][0]                  
__________________________________________________________________________________________________
conv2d_17 

In [ ]:
model.fit(train_ds, batch_size=None, epochs=60, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=36)

Epoch 37/60
173/174 [============================>.] - ETA: 0s - loss: 292.0950 - acc: 0.5795
Epoch 00037: loss did not improve from 275.38028
174/174 [==============================] - 19s 107ms/step - loss: 292.1248 - acc: 0.5784
Epoch 38/60
173/174 [============================>.] - ETA: 0s - loss: 283.2902 - acc: 0.6067
Epoch 00038: loss did not improve from 275.38028
174/174 [==============================] - 17s 95ms/step - loss: 283.4426 - acc: 0.6054
Epoch 39/60
173/174 [============================>.] - ETA: 0s - loss: 280.2272 - acc: 0.6049
Epoch 00039: loss did not improve from 275.38028
174/174 [==============================] - 17s 98ms/step - loss: 280.3406 - acc: 0.6039
Epoch 40/60
173/174 [============================>.] - ETA: 0s - loss: 279.3548 - acc: 0.6074
Epoch 00040: loss did not improve from 275.38028
174/174 [==============================] - 17s 97ms/step - loss: 279.4935 - acc: 0.6068
Epoch 41/60
173/174 [============================>.] - ETA: 0s - loss: 277.

In [ ]:
model.save('/content/model.hdf5')

In [ ]:
model.fit(train_ds, batch_size=None, epochs=200, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=60)

Epoch 61/200
173/174 [============================>.] - ETA: 0s - loss: 259.4795 - acc: 0.6257
Epoch 00061: loss improved from 261.07936 to 259.46978, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.61-259.47.h5
174/174 [==============================] - 17s 97ms/step - loss: 259.4698 - acc: 0.6253
Epoch 62/200
173/174 [============================>.] - ETA: 0s - loss: 258.5724 - acc: 0.6248
Epoch 00062: loss improved from 259.46978 to 258.82616, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.62-258.83.h5
174/174 [==============================] - 17s 99ms/step - loss: 258.8262 - acc: 0.6247
Epoch 63/200
173/174 [============================>.] - ETA: 0s - loss: 257.9906 - acc: 0.6270
Epoch 00063: loss improved from 258.82616 to 258.17788, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.63-258.18.h5
174/174 [==============================] - 17s 98ms/step - loss: 258.1779 - acc: 0.6269
Epoch 64/200
173/174 [======

In [ ]:
model.save('/content/gdrive/My Drive/MSR_Dataset/model/model_x64_epoch200_36.h5')

In [ ]:
model.fit(train_ds, batch_size=None, epochs=400, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=200)

Epoch 201/400
173/174 [============================>.] - ETA: 0s - loss: 220.5024 - acc: 0.6512
Epoch 00201: loss improved from 220.67825 to 220.60089, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.201-220.60.h5
174/174 [==============================] - 17s 100ms/step - loss: 220.6009 - acc: 0.6510
Epoch 202/400
173/174 [============================>.] - ETA: 0s - loss: 220.3747 - acc: 0.6529
Epoch 00202: loss improved from 220.60089 to 220.43162, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.202-220.43.h5
174/174 [==============================] - 17s 99ms/step - loss: 220.4316 - acc: 0.6524
Epoch 203/400
173/174 [============================>.] - ETA: 0s - loss: 220.2768 - acc: 0.6528
Epoch 00203: loss did not improve from 220.43162
174/174 [==============================] - 17s 96ms/step - loss: 220.4480 - acc: 0.6526
Epoch 204/400
173/174 [============================>.] - ETA: 0s - loss: 220.1236 - acc: 0.6527
Epoch 00204: loss 

In [ ]:
model.save('/content/gdrive/My Drive/MSR_Dataset/model/model_x64_epoch400_200.h5')

In [ ]:
model.load_weights('/content/gdrive/My Drive/MSR_Dataset/model/model_x64_epoch400_200.h5')

In [ ]:
model.fit(train_ds, batch_size=None, epochs=1000, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=400)

Epoch 401/1000
173/174 [============================>.] - ETA: 0s - loss: 203.2699 - acc: 0.6666
Epoch 00401: loss improved from 204.61477 to 203.47414, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.401-203.47.h5
174/174 [==============================] - 18s 104ms/step - loss: 203.4741 - acc: 0.6663
Epoch 402/1000
173/174 [============================>.] - ETA: 0s - loss: 204.2358 - acc: 0.6670
Epoch 00402: loss did not improve from 203.47414
174/174 [==============================] - 17s 100ms/step - loss: 204.2277 - acc: 0.6670
Epoch 403/1000
173/174 [============================>.] - ETA: 0s - loss: 204.4131 - acc: 0.6673
Epoch 00403: loss did not improve from 203.47414
174/174 [==============================] - 17s 97ms/step - loss: 204.2380 - acc: 0.6672
Epoch 404/1000
173/174 [============================>.] - ETA: 0s - loss: 204.4246 - acc: 0.6675
Epoch 00404: loss did not improve from 203.47414
174/174 [==============================] - 17s 96ms/step 

In [ ]:
model.load_weights('/content/gdrive/My Drive/MSR_Dataset/models_x64/weights.757-193.39.h5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 416         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 867         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 3 896         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [ ]:
model.fit(train_ds, batch_size=None, epochs=1500, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=757)

Instructions for updating:
Use tf.cast instead.
Epoch 758/1500
173/174 [============================>.] - ETA: 0s - loss: 221.9680 - acc: 0.6558
Epoch 00758: loss improved from inf to 222.33677, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.758-222.34.h5
174/174 [==============================] - 39s 223ms/step - loss: 222.3368 - acc: 0.6551
Epoch 759/1500
173/174 [============================>.] - ETA: 0s - loss: 232.2873 - acc: 0.6405
Epoch 00759: loss did not improve from 222.33677
174/174 [==============================] - 31s 175ms/step - loss: 232.3851 - acc: 0.6407
Epoch 760/1500
173/174 [============================>.] - ETA: 0s - loss: 224.7748 - acc: 0.6608
Epoch 00760: loss did not improve from 222.33677
174/174 [==============================] - 31s 176ms/step - loss: 224.7952 - acc: 0.6608
Epoch 761/1500
173/174 [============================>.] - ETA: 0s - loss: 228.6952 - acc: 0.6556
Epoch 00761: loss did not improve from 222.33677
174/174 [=====

In [ ]:
model.load_weights('/content/gdrive/My Drive/MSR_Dataset/models_x64/weights.1034-190.67.h5')

In [ ]:
model.fit(train_ds, batch_size=None, epochs=1500, steps_per_epoch=int(2*image_count/Batch_size+1), callbacks=callback_list, verbose=1, initial_epoch=1034)

Epoch 1035/1500
173/174 [============================>.] - ETA: 0s - loss: 212.4860 - acc: 0.6586
Epoch 01035: loss improved from inf to 212.54585, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.1035-212.55.h5
174/174 [==============================] - 35s 202ms/step - loss: 212.5458 - acc: 0.6584
Epoch 1036/1500
173/174 [============================>.] - ETA: 0s - loss: 210.9395 - acc: 0.6632
Epoch 01036: loss improved from 212.54585 to 211.03516, saving model to /content/gdrive/My Drive/MSR_Dataset/models_x64/weights.1036-211.04.h5
174/174 [==============================] - 31s 179ms/step - loss: 211.0352 - acc: 0.6631
Epoch 1037/1500
173/174 [============================>.] - ETA: 0s - loss: 211.5516 - acc: 0.6626
Epoch 01037: loss did not improve from 211.03516
174/174 [==============================] - 31s 178ms/step - loss: 211.6444 - acc: 0.6625
Epoch 1038/1500
 66/174 [==========>...................] - ETA: 19s - loss: 221.0530 - acc: 0.6994

KeyboardInterrupt: ignored